In [1]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
    #converting inventory rows to list of tuples from string
product = "Biologische prei"
filtered_transactions = transactions[transactions["description"] == product]
filtered_inventory = inventory[["day", "before or after delivery", product]]

## Testing `waste analysis` function 

In [2]:
complimentary = "Biologisch volkoren heel"
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=50)
output_full = waste_analysis(inventory, transactions, products, product, discount_per_day=discount_per_day,compl_prod=complimentary)
1.8624454296986455
output_full

Extended waste dataframe created
compl_product is a string
Data prepared for prediction
Model fitted; test set metrics: MAE: 1.8800259673437245 MSE: 5.048821404639766, R^2 score: -0.290824821125778
Demand function fitted
Value error occurred
Value error occurred
Value error occurred
Average of actual/estimated purchase ratios: 0.21296535558961688
Waste change predicted, calculating revenue loss...


,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn,predicted demand,demand_discounts,avg_price,std_price,expected purchases,expected waste,expected loss revenue,expected loss profit,expected waste cost,loss revenue,loss profit,waste cost
best before day,,,,,,,,,,,,,,,,,,,,
14,30,14.0,76.0,NaN,-46.0,-46.0,3,0.0,245,245,0.598816,0.597857,NaN,0.0,0.00,0.000000,0.00,0.00,0.00,0.00
17,30,17.0,85.0,9.0,21.0,-25.0,3,0.0,78,84,0.644000,0.690000,10.0,0.0,0.00,0.000000,0.00,0.00,0.00,0.00
21,25,21.0,96.0,11.0,14.0,-11.0,4,0.0,59,65,0.575000,0.690000,12.0,0.0,0.00,0.000000,0.00,0.00,0.00,0.00
24,25,24.0,108.0,12.0,13.0,2.0,4,2.0,60,68,0.472385,0.690000,13.0,1.0,0.47,0.322385,0.15,1.38,1.08,0.30
28,25,28.0,127.0,19.0,6.0,6.0,5,6.0,92,92,0.690000,0.690000,20.0,5.0,3.45,2.700000,0.75,4.14,3.24,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,25,350.0,1373.0,10.0,15.0,15.0,51,15.0,67,73,0.627273,0.690000,11.0,14.0,8.78,6.681818,2.10,10.35,8.10,2.25
353,25,352.0,1380.0,7.0,18.0,18.0,51,18.0,36,36,0.690000,0.690000,7.0,18.0,12.42,9.720000,2.70,12.42,9.72,2.70
357,25,357.0,1397.0,17.0,8.0,8.0,52,8.0,76,82,0.616941,0.690000,18.0,7.0,4.32,3.268588,1.05,5.52,4.32,1.20


In [3]:
money_cols = ["loss revenue", "loss profit", "waste cost", "expected loss revenue", "expected loss profit", "expected waste cost"]
output_full[money_cols].sum().values.round(2)

array([727.29, 567.24, 160.05, 535.03, 401.27, 133.8 ])

In [4]:
results = pd.DataFrame({"Metric":["Loss revenue", "Loss profit", "Waste cost","Loss revenue", "Loss profit", "Waste cost"], "Method":["Actual", "Actual", "Actual", "Predicted w/discounts", "Predicted w/discounts", "Predicted w/discounts"], "values":output_full[money_cols].sum().values.round(1)})
results["product"] = ["name", "name", "name", "name", "name", "name"]
results

,Metric,Method,values,product
0,Loss revenue,Actual,727.3,name
1,Loss profit,Actual,567.2,name
2,Waste cost,Actual,160.1,name
3,Loss revenue,Predicted w/discounts,535.0,name
4,Loss profit,Predicted w/discounts,401.3,name
5,Waste cost,Predicted w/discounts,133.8,name


In [5]:
import plotly.express as px
fig = px.bar(results, x="Metric", y='values', color="Method", barmode='group', text="values")
fig.show()

In [6]:
import plotly.express as px

output_copy = output_full.copy()
output_copy["MA7"] = output_copy['expected loss revenue'].rolling(7).mean()
output_copy["MA7_exp"] = output_copy["loss revenue"].rolling(7).mean()
fig = px.line(output_copy, x="DOY", y="MA7")
fig.add_scatter(x=output_copy["DOY"], y=output_copy["MA7_exp"], mode="lines")
fig.show()

In [2]:
from waste_functions import create_bb_data, add_purchases

waste_df = create_bb_data(filtered_inventory, product)
waste_df_extended = add_purchases(waste_df, transactions, product)
waste_df_extended

(101, 5)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\waste_functions.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["DOY"] = filtered_transactions.apply(input_DOY, axis=1)
C:\Users\zolyo\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn
best before day,,,,,,,,
14,26,14.0,36.0,NaN,-10.0,-10.0,3,0.0
17,26,17.0,41.0,5.0,21.0,11.0,3,11.0
21,26,21.0,60.0,19.0,7.0,7.0,4,7.0
24,26,24.0,68.0,8.0,18.0,18.0,4,18.0
28,26,28.0,88.0,20.0,6.0,6.0,5,6.0
...,...,...,...,...,...,...,...,...
350,26,350.0,1229.0,11.0,15.0,15.0,51,15.0
353,26,353.0,1237.0,8.0,18.0,18.0,51,18.0
357,26,357.0,1251.0,14.0,12.0,12.0,52,12.0


## Creating range intervals for mid-delivery sales calclation

In [3]:
best_before = []
for element_list in inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = list(np.unique(best_before))
#list of each unique best before dates
bb_dates.insert(0, 0)
ranges = []
i = 0
for item in bb_dates:
    if i < len(bb_dates)-1:
        i += 1
        ret_tupl = (item, bb_dates[i])
        ranges.append(ret_tupl)



In [4]:
ranges[4]

c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["day"] = pd.to_datetime(filtered_transactions["day"], dayfirst=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["discount"] = 100-(
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:29: SettingWith

In [30]:
from demand import *
compl_prod="Basis anannasstukjes"
discount_per_day = pd.read_csv("nr_discounts_per_day.csv")
prepared_transactions = prepare_data(filtered_transactions, discounts_per_day=discount_per_day)
output_dct = prepare_demand_function(prepared_transactions)

In [39]:
last = len(prepared_transactions)-1
pd.DataFrame(prepared_transactions.values, columns=prepared_transactions.columns).loc[last-1:]

,count,discount,std,min,25%,50%,75%,max,month,product,DOY,dayofweek,purchase_price,std_sales_price,prev_day_purchases,days,on_discount
346,2,0,0,0,0,0,0,0,December,Biologische knoflook,362,Saturday,0.69,0.69,4,29-12-2018,12
347,2,0,0,0,0,0,0,0,December,Biologische knoflook,363,Sunday,0.69,0.69,2,30-12-2018,13


In [6]:
model = fit_demand_function(output_dct)

Model fitted; test set metrics: MAE: 1.4358902314077937 MSE: 3.326587120366766, R^2 score: -0.11815451624990914


In [6]:
from waste_functions import predicted_demand

test = predicted_demand(waste_df_extended, ranges, model, output_dct, prepared_transactions)

as.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.dropna(inplace=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.dropna(inplace=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.dropna(inplace=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
test

,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn,predicted demand,demand_discounts
best before day,,,,,,,,,,
14,26,14.0,36.0,NaN,-10.0,-10.0,3,0.0,38.803361,40.901873
17,26,17.0,41.0,5.0,21.0,11.0,3,11.0,7.904251,10.002763
21,26,21.0,60.0,19.0,7.0,7.0,4,7.0,12.064346,14.162858
24,26,24.0,68.0,8.0,18.0,18.0,4,18.0,12.743521,14.842033
28,26,28.0,88.0,20.0,6.0,6.0,5,6.0,22.054773,24.153285
...,...,...,...,...,...,...,...,...,...,...
350,26,350.0,1229.0,11.0,15.0,15.0,51,15.0,21.842744,23.941256
353,26,353.0,1237.0,8.0,18.0,18.0,51,18.0,12.500279,14.598791
357,26,357.0,1251.0,14.0,12.0,12.0,52,12.0,21.619144,23.717656
